**Background:**

ORIE 4741 uses peer grading to determine scores on final projects. Each project has an underlying quality; some are good, some less good. Some students are fair graders, and report the project quality as their grade. Some are easy graders, and report a higher grade. Some are harsh graders, and report a lower grade.
As a result, some students fear that peer grading is unfair: why should their grade suffer simply through the random chance of a harsh reviewer?

An ideal solution might be to have every student grade every project. 
However, this solution is rarely popular with reviewers.
Instead, in this homework problem, we will explore whether we can predict the ratings 
that all other reviewers *would have* given had they reviewed all projects. 
We will try this technique both on a synthetic data and on the real peer-review rating scores for ORIE4741 projects in 2017.

Formally, let's define our problem. 
There are $d$ students enrolled in ORIE 4741 who have formed $n$ project groups.
Each student is responsible for grading $p$ projects. 
(In our class, $p$=2.)
We'll collect the grades into a grade matrix $A \in \mathbb{R}^{n \times d}$: 
$A_{ij}$ will represent the grade that student $j$ would assign to project $i$.

Of course, we cannot assign each student to grade every project. Instead, we make peer review assignments $\Omega = \{(i_1, j_1), \ldots, \}$. Here, $(i,j) \in \Omega$ if student $j$ is assigned to grade project $i$. 

Unfortunately, this means that some projects are assigned harder graders than other projects. Our goal is to find a fair way to compute a project's final grade. We consider two methods:

1.  *Averaging.* The grade $g_i$ for project $i$ is the average of the grades given by peer reviewers:

    $$g^\text{avg}_i = \frac 1 p \sum_{j: (i,j) \in \Omega} A_{ij}$$

2.  *Matrix completion.* We fit a low rank model to the grade matrix and use it to compute an estimate $\hat A$ of the grade matrix. We will try a few different losses $\ell$ and regularizers $r$ to see which work best.
<!--    To be more concrete, let's suppose that we find $\hat A$ by fitting a rank $k$ model. We will use Huber loss, for robustness against outlier grades, and nonnegative regularization, since both student grading toughness and project quality are nonnegative. -->
    $$\text{minimize} \quad \sum_{(i,j) \in \Omega} \ell(A_{ij} - (X^T Y)_{ij}) + r(X) + r(Y), $$
    where $X \in \mathbb{R}^{k \times n}$ and $Y \in \mathbb{R}^{k \times d}$. 
    
    We will then compute our estimate $\hat A$ as 
    $$ \hat A = X^T Y. $$
    In other words, $\hat A$ is the rank-$k$ matrix that matches the observations best in the sense of Huber error.
    
    We compute the grade $g_i$ for project $i$ as the average of these estimated grades:

    $$g^\text{mc}_i = \frac 1 n \sum_{j=1}^n \hat A_{ij}$$

In this problem, we will consider which of these two grading schemes, averaging or matrix completion, is better.

In [ ]:
using LowRankModels, Random, LinearAlgebra, Plots, Statistics, CSV

Note: Throughout the notebook, we will be using random seeds for reproducibility. Do not remove the corresponding commands.

## (a) Synthetic data

We first take a look at a demo. We generate a synthetic dataset that contains ratings with integer values 1-5. 1 is lowest and 5 is highest. The rows are aligned by projects; the columns are aligned by reviewers.

In [ ]:
# rows are papers, columns are reviewers
Random.seed!(1)  # seed random seed, for reproducibility

# problem dimensions
n,d = 100,200    # n projects, d reviewers

# observed entries
Ω = findall(rand(n,d) .< .1)  # Ω is a matrix with the same shape as our 10% of entries are observed. 

# latent parameters 
θ = rand(n)      # quality of paper 
a = rand(d)      # coefficient of reviewer
b = rand(d)      # offset of reviewer

# data matrix
A = θ * a' .+ b'

a = vec(A)
t = [quantile(a,.22), quantile(a,.31), quantile(a,.53), quantile(a,.88)] # thresholds

# generate ratings matrix
R = ones(Int, n, d)
for ti in t
    R += A .> ti
end

In [ ]:
# We can plot histogram of scores
histogram(vec(R))

In [ ]:
# compute observed average scores on all projects
observed_average_score = []
for i=1:n
    append!(observed_average_score, mean(R[filter(t -> t[1] == i, Ω)]))
end

In [ ]:
# compute true average scores on all projects
true_average_score = mean(R, dims=2);

Next, we take a look at different matrix completion scenarios.

## Fit ratings with Quadratic Loss

We first fit a low rank model to this simulated data using quadratic loss and regularizers.

In [ ]:
Random.seed!(1)      
loss = QuadLoss()    # quadratic loss
reg = QuadReg(.0001) # a tiny bit of quadratic regularization
k = 2                # we'll add the offset separately below 
glrm = GLRM(R, loss, reg, reg, k, obs=Ω)  # the GLRM object stores the model, data, and parameter estimate
add_offset!(glrm)    # adds an offset to the model

# fit low rank model
fit!(glrm)                            # alternating minimization
Rhat_quad = impute(glrm)                   # imputed values 
MAE_quad = sum(abs.(vec(Rhat_quad - R)))/(n*d)  # mean absolute error
@show MAE_quad;

To evaluate the performance of our predictions, we define the following terms:

1. observed average score: the average of all scores we observe on a single project.
2. true average score: the average of all scores on a single project, whether we observed or not. These scores are regarded as true labels.
3. predicted score: the average of all predicted scores on a single project.

In [ ]:
# predicted scores on all projects
pred_score_quad = mean(Rhat_quad, dims=2);

**Question**: 

Plot histograms of
1. the difference between the observed average score and the true average score
2. the difference between the predicted average score and the true average score

Notice that for real data, the true average score is not known, while the observed average score is known and the predicted average score is computable.
Which predicts the true average score best? Compare the variance of the errors. 

In [ ]:
"YOUR WORK HERE: histograms"


In [ ]:
"YOUR WORK HERE: variance comparison"


## Fit with the BvSLoss function

Notice that ratings are ordinal, taking values between 1 and 5. 
Hence it makes more sense to use an ordinal loss. 
We will use BvSLoss (Bigger-versus-Smaller) here. 
As we saw in class, using the BvSLoss is equivalent to mapping the matrix through the ordinal embedding
    
    1 => [0, 0, 0, 0]
    2 => [1, 0, 0, 0]
    3 => [1, 1, 0, 0]
    4 => [1, 1, 1, 0]
    5 => [1, 1, 1, 1]
    
 and fitting the resulting matrix with the hinge loss or logistic loss.
 
 Fit a model to the data using the BvSLoss and a nonnegative regularizer.

In [ ]:
# form a low rank model by selecting appropriate losses, regularizer, and rank k
loss = BvSLoss(5, bin_loss=LogisticLoss())    # BvSLoss with 5 levels, using Logistic Loss as the underlying binary loss
# you could also ask for nonnegative coefficients:
rx = lastentry1(NonNegConstraint())
ry = OrdinalReg(NonNegConstraint())
# rx = lastentry1(QuadReg(.0001))
# ry = OrdinalReg(QuadReg(.0001))
k = 2                # we'll add the offset separately below 
glrm = GLRM(R, loss, rx, ry, k, obs=Ω)  # the GLRM object stores the model, data, and parameter estimate

# initialize with random positive numbers
glrm.X = rand(size(glrm.X)...)
glrm.Y = rand(size(glrm.Y)...)

# fit low rank model 
fit!(glrm)                            # alternating minimization
Rhat_bvs = impute(glrm)                   # imputed values 
MAE_bvs = sum(abs.(vec(Rhat_bvs - R)))/(n*d)  # mean absolute error
@show MAE_bvs;

In [ ]:
pred_score_bvs = mean(Rhat_bvs, dims=2);

**Question**: 

Plot a histogram of the error between the predicted score using the BvSLoss and the true average score.

Compare the accuracy of the predictions from the model fit with BvSLoss and QuadLoss. Which predicts the ratings better (say, in mean absolute error)? Which predicts the true average score better (say, in mean absolute error)? 

In [ ]:
"YOUR WORK HERE: histograms"

In [ ]:
"YOUR WORK HERE: accuracy comparison by, e.g., mean absolute error"

**Question**: 

Fit at least one more model to this dataset. How did you choose which loss or regularizers to use? How does the error of your method compare to the error of the two models we fit above?

The ORIE 4741 must choose a method to grade your projects based on observable data. Based on these results on synthetic data, how would you like them to grade you? (Note: you'll have a chance to answer again after you see the real data.)

1. average the observed scores 
2. average the scores imputed by matrix completion with quadratic loss
3. average the scores imputed by matrix completion with BvSLoss
4. average the scores imputed by matrix completion with my model

In [ ]:
"YOUR ANSWER HERE"

# (b) Fall 2017 ORIE4741 project review data

Now let's see how well matrix completion works on the real data!
The rating scores are ordinal and have 6 possible values.
Here, we don't have access to the true average score. 

In [ ]:
ratings = CSV.read("ratings.csv");

In [ ]:
staff_score = ratings[:, 2]; # rating scores given by staff
average_score = ratings[:, 3]; # rating scores from averages of peer review scores
R_real = convert(Matrix, ratings[:, 4:end]); # peer review grades
Ω_real = findall(.!ismissing.(R_real)); # observed entries
n_real, p_real = size(R_real);

First, let's plot the average scores

In [ ]:
histogram(average_score, nbins=20)

## Fit ratings with several losses

First, we choose Huber loss with nonnegative constraint.

In [ ]:
# form a low rank model by selecting appropriate losses, regularizer, and rank k
Random.seed!(1)
loss = HuberLoss() 
reg = NonNegConstraint() 
k = 2                # we'll add the offset separately below 
glrm = GLRM(R_real, loss, reg, reg, k, obs=Ω_real)
add_offset!(glrm)    # adds an offset to the model

# fit low rank model 
fit!(glrm)                            # alternating minimization
Rhat_real_huber = impute(glrm)                   # imputed values 
MAE_real_huber = sum(abs.(vec((Rhat_real_huber - R_real)[Ω_real])))/(n_real*p_real)  # mean absolute error
pred_score_quad = mean(Rhat_real_huber, dims=2)  # predicted scores of each project
@show MAE_real_huber;

In [ ]:
# We plot the predictions
histogram(pred_score_quad, nbins=20)

Now let's see how the L1 loss performs.

In [ ]:
# form a low rank model by selecting appropriate losses, regularizer, and rank k
Random.seed!(1)
loss = L1Loss()    # BvSLoss with 5 levels
reg = QuadReg(.0001) # a tiny bit of quadratic regularization
k = 2                # we'll add the offset separately below 
glrm = GLRM(R_real, loss, reg, reg, k, obs=Ω_real)  # the GLRM object stores the model, data, and parameter estimate
add_offset!(glrm)    # adds an offset to the model

# fit low rank model
fit!(glrm)                            # alternating minimization
Rhat_real_l1 = impute(glrm)                   # imputed values 
MAE_real_l1 = sum(abs.(vec((Rhat_real_l1 - R_real)[Ω_real])))/(n_real*p_real)  # mean absolute error
@show MAE_real_l1;

In [ ]:
pred_score_l1 = mean(Rhat_real_l1, dims=2)
histogram(pred_score_l1, nbins=20)

## Fit ratings with BvSLoss

For convenience in fitting the BvSLoss, we create a copy of the peer grades with missing entries imputed by 0. This won't affect the fit of the GLRM, since we restrict it to look only at the observed entries.

In [ ]:
R_real_noNA = copy(R_real);
for row in 1:n_real
      for col in 1:p_real
        if  ismissing(R_real[row, col])
            R_real_noNA[row, col] = 0
        end
      end
end
R_real_noNA = Int.(R_real_noNA);

In [ ]:
Random.seed!(1)
loss = BvSLoss(6, bin_loss=LogisticLoss())    # BvSLoss with 6 levels, using Logistic Loss as the underlying binary loss
# could also ask for nonnegative coefficients:
rx = lastentry1(QuadReg(.0001))
ry = OrdinalReg(QuadReg(.0001))

# rx = lastentry1(NonNegConstraint())
# ry = OrdinalReg(NonNegConstraint())
k = 2                # we'll add the offset separately below 
glrm = GLRM(R_real_noNA, loss, rx, ry, k, obs=Ω_real)  # the GLRM object stores the model, data, and parameter estimate

# initialize with random positive numbers
glrm.X = rand(size(glrm.X)...)
glrm.Y = rand(size(glrm.Y)...)

# fit low rank model 
fit!(glrm)                            # alternating minimization
Rhat_real_bvs = impute(glrm)                   # imputed values 
MAE_real_bvs = sum(abs.(vec((Rhat_real_bvs - R_real)[Ω_real])))/(n_real*p_real)  # mean absolute error
@show MAE_real_bvs;

In [ ]:
pred_score_bvs = mean(Rhat_real_bvs .- 0, dims=2);
histogram(pred_score_bvs, nbins = 20)

**Question**: 

Fit at least one more model to this dataset. How did you choose which loss or regularizers to use? How does the error of your method compare to the error of the two models we fit above?

In [ ]:
"YOUR WORK HERE: more model fitting"

**Question**: Comment on the performance of all of these models on real data. 
Your answer should discuss how the prediction errors are distributed.

In [ ]:
"YOUR ANSWER HERE"

## (c) Summary

The ORIE 4741 must choose a method to grade your projects based on observable data. How would you like them to grade you?
1. average the observed scores 
2. average the scores imputed by matrix completion with quadratic loss
3. average the scores imputed by matrix completion with BvSLoss
4. average the scores imputed by matrix completion with my model

**Question**: Do you prefer simple grade averaging or matrix completion? If you prefer matrix completion, state what loss and regularizer you'd like the course staff to use (the polling result may affect your staff's choice!).

In [ ]:
"YOUR ANSWER HERE"